# Housekeeping

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import sqlite3

from jsonschema.benchmarks.const_vs_enum import value
from pandas.tseries.offsets import *
from datetime import timedelta
from datetime import datetime
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')

today = dt.datetime.today().strftime('%Y-%m-%d')
today

'2026-01-08'

# Download WRDS data - as of January 8, 2026

In [2]:
import wrds
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
pgpass file created at C:\Users\jessf\AppData\Roaming\postgresql\pgpass.conf
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
count = db.get_row_count('comp', 'funda')
count

928546

In [4]:
funda = db.raw_sql("SELECT * FROM comp.funda WHERE indfmt='INDL' AND datafmt = 'STD' AND popsrc = 'D' AND consol = 'C'", date_cols=['datadate'])
len(funda)

593165

# Calculate firm age

In [10]:
# Calculate firm age as of each fiscal year
firm_first_year = funda.groupby('gvkey')['fyear'].min().reset_index()
firm_first_year.columns = ['gvkey', 'first_year']

# Merge back to main dataframe
funda1 = funda.merge(firm_first_year, on='gvkey', how='left')

# Calculate age as of each fiscal year
funda1['firm_age'] = funda1['fyear'] - funda1['first_year'] + 1

In [11]:
funda1.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,cusip,conm,...,prcl_f,adjex_f,rank,au,auop,auopic,ceoso,cfoso,first_year,firm_age
0,001000,1961-12-31,1961,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,<NA>,3.341831,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1961,1
1,001000,1962-12-31,1962,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,<NA>,3.341831,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1961,2
2,001000,1963-12-31,1963,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,<NA>,3.244497,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1961,3
3,001000,1964-12-31,1964,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,<NA>,3.089999,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1961,4
4,001000,1965-12-31,1965,INDL,C,D,STD,AE.2,000032102,A & E PLASTIK PAK INC,...,<NA>,3.089999,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1961,5


In [12]:
funda1['ln_age'] = np.log(funda1['firm_age'])

In [13]:
funda1.to_pickle(f"../Data/analysis/compustat_firm_age_{today}.pkl")